In [1]:
import csv
import glob
import numpy as np
import os
import pandas as pd
import subprocess
import sys

BUILD = "hg19"
PATH = f"/dors/capra_lab/data/ucsc/{BUILD}/synteny_age_{BUILD}"
CHR = sys.argv[1]

# drop non-important columns and collapse coordinates by MRCA. 

# Functions 

In [2]:
def make_chr_list():
    n = list(np.arange(1, 23))
    # n.append("X")

    chr_list = []

    for num in n:
        chrn = "chr" + str(num)
        chr_list.append(chrn)

    return chr_list


def drop(f, chr_):
    os.chdir(os.path.join(PATH, "summarized"))
    cutf = os.path.join(PATH, "summarized", f"cut_{chr_}_syn_age.bed")
    cmd = f"cut -f 1,2,3,8 {f} > {cutf}"
    subprocess.call(cmd, shell = True)
    
    return cutf

def lump(F):

    # parse through rows, consolidate coordinates of the same age. 
    start, stop = 0, 0
    last_mrca = -1

    results = {}
    with open(cut_f, "r") as df:
        for i, row in enumerate(csv.reader(df, delimiter='\t')):

            chr_, start_, stop_ = row[0], row[1], row[2]
            mrca = row[3]

            if last_mrca == -1:  # for the first row of the segment
                start = start_
                stop = stop_
                last_mrca = mrca

            elif mrca == last_mrca:  # still in the same MRCA
                stop = stop_  # add to the stop

            elif mrca!= last_mrca:
                new_row = [chr_, start, stop, last_mrca]
                results[i] = new_row

                # reset values
                start = start_
                stop = stop_
                last_mrca = mrca

    return results

# Stack rows, make dataframe. 

def stack(results):
    
    data = np.array(["#chr", "start", "stop", "mrca"])
    for val in results.values():
        a = np.array(val)
        if "chr" not in a[0]
        a = a[1:]
        data = np.vstack((data,a,))

    re = pd.DataFrame(data)
    header_row = 0

    re.columns = re.iloc[header_row]
    re = re.drop(header_row).reset_index()
    re["mrca"] = re["mrca"].map(float).round(3)
    re = re.drop(["index"], axis = 1) # drop the index column
    return re

# in case you need to split file on size before getting started
def split_by_line(f, data_path, chr_num):

    chr_path = os.path.join(data_path, chr_num)
    try:
        os.mkdir(chr_path)
    except FileExistsError:
        pass

    # change dir to the path (I think I did this already, but incase)
    os.chdir(chr_path)

    # split the file in command line into sizes of 1000 lines
    cmd = f"split -l 1000 {f} {chr_num}-"

    subprocess.call(cmd, shell = True)

    small_fs = glob.glob(f"{chr_path}/{chr_num}-*")


    return small_fs


# make list of chromosomes

In [3]:
chr_list = make_chr_list()

# unzip and cut syntenic file into 4 columns 

In [46]:
for CHR in chr_list:
    Fzipped = os.path.join(PATH, f"{CHR}_syn_age.bed.gz")

    F = os.path.join(PATH, f"{CHR}_syn_age.bed")

    RE = os.path.join(PATH, "summarized",  f"cut_{CHR}_syn_age.bed")

    if os.path.exists(RE) is False:
        print(RE)
        # unzip
        cmd = f"gunzip {Fzipped}"
        if os.path.exists(Fzipped) is True:
            subprocess.call(cmd, shell = True)

        cut_f = drop(F, CHR)
        
        cmd = f"gzip {F}"
        if os.path.exists(Fzipped) is False:
            subprocess.call(cmd, shell = True)

# split by line

In [47]:
for CHR in chr_list:

    chr_f = os.path.join(PATH, "summarized", f"cut_{CHR}_syn_age.bed")

    # test whether you've split chr file into 1000 line files.
    chr_path = os.path.join(PATH,"summarized")
    test_file = os.path.join(chr_path, f"{CHR}-aa")

    if os.path.exists(test_file) is False:

        splits = split_by_line(chr_f, chr_path, CHR)

# lump coordinates by sequence age 

In [48]:
for CHR in chr_list:
    print(CHR)
    cut_fs = glob.glob(os.path.join(PATH, "summarized", CHR,  f"{CHR}-*"))
    RE = os.path.join(PATH, "summarized", f"{CHR}_syn_age.bed")
    for n, cut_f in enumerate(cut_fs):
        id_ = cut_f.split(CHR)[-1]

        results = lump(cut_f)
        
        s = stack(results)
        s.to_csv(RE, sep = '\t', index = False, mode = "a")

        os.remove(cut_f)

chr1
chr2
chr3
chr4
chr5
chr6
chr7
chr8
chr9
chr10
chr11
chr12
chr13
chr14
chr15
chr16
chr17
chr18
chr19
chr20
chr21
chr22
